In [67]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [68]:
pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda install pytorch torchvision -c pytorch

In [ ]:
import torch 
import torch.nn as nn

In [ ]:
df= pd.read_csv("AAPL.csv")

In [ ]:
closed_prices=df['Close']

In [ ]:
seq_len =15

In [ ]:
mm =MinMaxScaler()

In [ ]:
scaler_price =mm.fit_transform(np.array(closed_prices)[... ,None]).squeeze()

In [ ]:
x=[]
y=[]


In [ ]:
for i in range(len(scaler_price)-seq_len):
    x.append(scaler_price[i : i+seq_len])
    y.append(scaler_price[i+seq_len])

### split into train and test

In [ ]:
x =np.array(x)[... , None]
y =np.array(y)[... , None]

In [ ]:
train_x = torch.from_numpy(x[:int (0.8 * x.shape[0])]).float()

In [ ]:
train_y = torch.from_numpy(y[:int (0.8 * x.shape[0])]).float()

In [ ]:
test_x = torch.from_numpy(x[int (0.8 * x.shape[0]):]).float()

In [ ]:
test_y = torch.from_numpy(y[int (0.8 * x.shape[0]):]).float()

In [ ]:
class Model (nn.Module):
    def __init__(self , input_size , hidden_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size , hidden_size,batch_first = True)
        self.fc = nn.Linear(hidden_size, 1)
    def forward(self, x):
        output ,(hidden , cell ) = self.lstm(x)
        return self.fc(hidden[-1,:1])
model = Model (1 , 96)

optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
loss_fn = nn.MSELoss()

In [ ]:
num_epochs =100

for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(train_x)
    loss = loss_fn(output, train_y)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(epoch, "epoch loss", loss.item())
model.eval()
with torch.no_grad():
    test_output = model(test_x)

pred = mm.inverse_transform(test_output.numpy())
real = mm.inverse_transform(test_y.numpy())

plt.plot(pred.squeeze(), color='blue', label='predicted')
plt.plot(real.squeeze(), color='yellow', label='real')
plt.legend()
plt.show()
